In [1]:
import json
import re
from collections import Counter
import requests

In [9]:
# 20.JSONデータの読み込み
with open("./jawiki-country.json", "r") as f:
    line = f.readline()
    while line:
        wiki = json.loads(f.readline())
        if wiki["title"] == u"イギリス":
            line = None
print wiki["title"]

イギリス


In [3]:
# 21. カテゴリ名を含む行を抽出
pattern = re.compile("\[\[Category:.*\]\]\n")
cat_list = pattern.findall(wiki["text"])
for cat in cat_list:
    print cat

[[Category:イギリス|*]]

[[Category:英連邦王国|*]]

[[Category:G8加盟国]]

[[Category:欧州連合加盟国]]

[[Category:海洋国家]]

[[Category:君主国]]

[[Category:島国|くれいとふりてん]]



In [4]:
# 22.カテゴリ名の抽出
pattern = re.compile(":.*\||:.*\]")
cname_list = []
for cat in cat_list:
    m = pattern.search(cat)
    cname = cat[m.span()[0]+1: m.span()[1]-1].replace("]", "")
    cname_list.append(cname)
for cname in cname_list:
    print cname

イギリス
英連邦王国
G8加盟国
欧州連合加盟国
海洋国家
君主国
島国


In [6]:
# 23.セクション構造
pattern = re.compile("=[=]+.*=[=]+\n")
sec_list = pattern.findall(wiki["text"])
print("~~~~~~~~~~~~セクション構造~~~~~~~~~~~~")
for sec in sec_list:
    c = Counter(sec)
    print sec.replace("=", "").replace("\n", "") + ":" + str((c["="] / 2) - 1)

~~~~~~~~~~~~セクション構造~~~~~~~~~~~~
国名:1
歴史:1
地理:1
気候:2
政治:1
外交と軍事:1
地方行政区分:1
主要都市:2
科学技術:1
経済:1
鉱業:2
農業:2
貿易:2
通貨:2
企業:2
交通:1
道路:2
鉄道:2
海運:2
航空:2
通信:1
国民:1
言語:2
宗教:2
 婚姻 :2
教育:2
文化:1
食文化:2
文学:2
 哲学 :2
音楽:2
イギリスのポピュラー音楽:3
映画:2
コメディ:2
国花:2
世界遺産:2
祝祭日:2
スポーツ:1
サッカー:2
競馬:2
モータースポーツ:2
脚注:1
関連項目:1
外部リンク:1


In [7]:
# 24.ファイル参照の抽出
'''
=http://~ .htm or .html|
File:Elizaq ~ .jpg png| etc.....
ファイル; ~ .jpg|
<ref>[http://www.sh.xinhuanet.com/shstatics/images2013/IFCD2013_En.pdf Xinhua-Dow Jones International Financial Centers Development Index （2013)] (2013年9月公表)</ref>
'''
url = "https?://[-_.!~*\'();\/?:@&+=\$,%#a-zA-Z0-9]*[\s\|]{1}"
file_pattern = u"File:[a-zA-Z0-9\s]+\.[a-z]+\||ファイル:[a-zA-Z0-9\s]+\.[a-z]+\|"
pattern = re.compile(url + "|" + file_pattern)
medias = pattern.findall(wiki["text"])
media_list = []
for media in medias:
    if media.startswith(u"ファイル") or media.startswith(u"File"):
        media = media[5:len(media)-1]
        media_list.append(media)
    elif media.startswith(u"http"):
        if media[-1] == "|":
            media_list.append(media[0:len(media)-1])
        else:
            media_list.append(media)

for media in media_list:
    print media

Royal Coat of Arms of the United Kingdom.svg
http://esa.un.org/unpd/wpp/Excel-Data/population.htm 
http://www.imf.org/external/pubs/ft/weo/2012/02/weodata/weorept.aspx?pr.x=70&pr.y=13&sy=2010&ey=2012&scsm=1&ssd=1&sort=country&ds=.&br=1&c=112&s=NGDP%2CNGDPD%2CPPPGDP%2CPPPPC&grp=0&a= 
http://www.mod.go.jp/msdf/formal/info/news/200808/082001.html 
The British Empire.png
http://www.cnn.co.jp/world/35023094.html
Uk topo en.jpg
BenNevis2005.jpg
http://yoshio-kusano.sakura.ne.jp/nakayamakouen6newpage3.html 
David Cameron and Barack Obama at the G20 Summit in Toronto.jpg
http://www.globalpowereurope.eu/
http://www.raf.mod.uk/legalservices/p3chp29.htm
http://www.mod.uk/DefenceInternet/AboutDefence/Organisation/KeyFactsAboutDefence/DefenceSpending.htm
http://www.mod.uk/NR/rdonlyres/6FBA7459-7407-4B85-AA47-7063F1F22461/0/modara_0405_s1_resources.pdf 
http://www.asahi.com/international/update/1201/TKY201111300900.html?ref=reca 
http://mainichi.jp/select/world/europe/news/20111130k0000e030066000c.h

In [10]:
# 25.テンプレートの抽出
lines = wiki["text"].split("\n")
index = 0
infos = {}
rm_pattern = r"<ref.*/>|<ref.*</ref>"
line = lines[index]
while line:
    line = lines[index]
    if line.startswith("}}"):
        line = None
    elif line.startswith("|"):
        key, value = line[1:len(line)].split(" = ")
        infos[key] = re.sub(rm_pattern, "", value)
    index += 1
    
for key in infos:
    print key + ":" + infos[key]

人口値:63,181,775
国章リンク:（[[イギリスの国章|国章]]）
元首等氏名:[[エリザベス2世]]
時間帯:±0
標語:{{lang|fr|Dieu et mon droit}}<br/>（[[フランス語]]:神と私の権利）
面積順位:76
公式国名:{{lang|en|United Kingdom of Great Britain and Northern Ireland}}
注記:
面積値:244,820
通貨コード:GBP
位置画像:Location_UK_EU_Europe_001.svg
面積大きさ:1 E11
人口密度値:246
確立形態1:[[イングランド王国]]／[[スコットランド王国]]<br />（両国とも[[連合法 (1707年)|1707年連合法]]まで）
国章画像:[[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]
GDP統計年:2012
確立形態2:[[グレートブリテン王国]]建国<br />（[[連合法 (1707年)|1707年連合法]]）
建国形態:建国
GDP統計年MER:2012
GDP順位MER:5
首都:[[ロンドン]]
確立形態4:現在の国号「'''グレートブリテン及び北アイルランド連合王国'''」に変更
日本語国名:グレートブリテン及び北アイルランド連合王国
元首等肩書:[[イギリスの君主|女王]]
GDP順位:6
GDP値:2兆3162億
ccTLD:[[.uk]] / [[.gb]]
確立形態3:[[グレートブリテン及びアイルランド連合王国]]建国<br />（[[連合法 (1800年)|1800年連合法]]）
GDP統計年元:2012
公用語:[[英語]]（事実上）
国旗画像:Flag of the United Kingdom.svg
国際電話番号:44
夏時間:+1
最大都市:ロンドン
略名:イギリス
GDP値元:1兆5478億
GDP値MER:2兆4337億
水面積率:1.3%
首相等氏名:[[デーヴィッド・キャメロン]]
首相等肩書:[[イギリスの首相|首相]]
人口順位:22
GDP/人:36,727
人口統計年:2011
通貨:[[スターリング・ポンド|UKポンド]] (&pound;)
ISO 3166-1:

In [11]:
# 26.強調マークアップの除去&27.内部リンクの除去&28.MediaWikiマークアップの除去
def replace_markup(text):
    # 強調の除去
    em_pattern = r"'''''|'''|''"
    text = re.sub(em_pattern, "", text)
    markup_pattern = re.compile("\[\[.*?\]\]|\{\{.*?\}\}")
    match_list = markup_pattern.findall(text)
    cleaned_list = []
    for match in match_list:
        sep = match.split("|")
        # [[ロンドン]]
        if len(sep) == 1:
            cleaned_text = sep[0][2:len(sep[0])-2]
        # [[連合法 (1707年)|1707年連合法]]
        elif len(sep) == 2:
            cleaned_text = sep[0][2:]
        elif len(sep) == 3:
            # {{lang|en|United Kingdom of Great Britain and Northern Ireland}}
            if sep[0].startswith("{"):
                cleaned_text = sep[2][:len(sep[2])-2]
            # [[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]
            elif sep[0].startswith("["):
                cleaned_text = sep[0][7:]
        cleaned_list.append(cleaned_text)
    for index in range(len(match_list)):
        text = text.replace(match_list[index], cleaned_list[index])
    return text

for key in infos:
    print key + ":" + replace_markup(infos[key])

人口値:63,181,775
国章リンク:（イギリスの国章）
元首等氏名:エリザベス2世
時間帯:±0
標語:Dieu et mon droit<br/>（フランス語:神と私の権利）
面積順位:76
公式国名:United Kingdom of Great Britain and Northern Ireland
注記:
面積値:244,820
通貨コード:GBP
位置画像:Location_UK_EU_Europe_001.svg
面積大きさ:1 E11
人口密度値:246
確立形態1:イングランド王国／スコットランド王国<br />（両国とも連合法 (1707年)まで）
国章画像:Royal Coat of Arms of the United Kingdom.svg
GDP統計年:2012
確立形態2:グレートブリテン王国建国<br />（連合法 (1707年)）
建国形態:建国
GDP統計年MER:2012
GDP順位MER:5
首都:ロンドン
確立形態4:現在の国号「グレートブリテン及び北アイルランド連合王国」に変更
日本語国名:グレートブリテン及び北アイルランド連合王国
元首等肩書:イギリスの君主
GDP順位:6
GDP値:2兆3162億
ccTLD:.uk / .gb
確立形態3:グレートブリテン及びアイルランド連合王国建国<br />（連合法 (1800年)）
GDP統計年元:2012
公用語:英語（事実上）
国旗画像:Flag of the United Kingdom.svg
国際電話番号:44
夏時間:+1
最大都市:ロンドン
略名:イギリス
GDP値元:1兆5478億
GDP値MER:2兆4337億
水面積率:1.3%
首相等氏名:デーヴィッド・キャメロン
首相等肩書:イギリスの首相
人口順位:22
GDP/人:36,727
人口統計年:2011
通貨:スターリング・ポンド (&pound;)
ISO 3166-1:GB / GBR
人口大きさ:1 E7
確立年月日4:1927年
国歌:女王陛下万歳
確立年月日1:927年／843年
確立年月日2:1707年
確立年月日3:1801年


In [12]:
# 29.国旗画像のURLを取得する
# 画像を取得するwikiのWebAPIいけてねぇな・・・
payload = {"action": "query", "prop": "imageinfo", "iiprop": "url", "format": "json", "titles": "File:" + infos[u"国旗画像"]}
response = requests.get("https://commons.wikimedia.org/w/api.php", params=payload)
result = json.loads(response.text)
print result["query"]["pages"]["347935"]["imageinfo"][0]["url"]

https://upload.wikimedia.org/wikipedia/commons/a/ae/Flag_of_the_United_Kingdom.svg
